# Campervan Projekt

## Libraries and settings

In [427]:
# Libraries
import os
import re
import sqlite3
import fnmatch
import pprint
import requests
import numpy as np
import pandas as pd

import geopandas as gpd

from shapely.geometry import Polygon

import matplotlib.patches as patches
from matplotlib import pyplot as plt

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Function to close a sqlite db-connection
def check_conn(conn):
     try:
        conn.cursor()
        return True
     except Exception as ex:
        return False


print(os.getcwd())

/workspaces/project_Autoscout_Coding_Pros/Project_Campers


## Import data

In [428]:
# Read data
df_orig = pd.read_csv("./data/Autoscout24_Camper.csv", sep=",", encoding='utf-8')

# Show column names from df_orig
print(df_orig.columns)

# define the columns to keep
colums = ['web-scraper-order',
        'description_raw',
        'price_raw', 
        'mileage_raw',
       'fuel_type_raw', 
       'location_raw']

# keep only the columns we are interested in
df = df_orig[colums]

# Show first records of data frame
df.head()


Index(['web-scraper-order', 'web-scraper-start-url', 'autoscout24',
       'autoscout24-href', 'description_raw', 'price_raw', 'mileage_raw',
       'fuel_type_raw', 'location_raw'],
      dtype='object')


,web-scraper-order,description_raw,price_raw,mileage_raw,fuel_type_raw,location_raw
0,1733470796-1,MERCEDES-BENZ Camper,CHF 15'900.–,265'000 km,Diesel,"Mittelstrasse 38, 3613 Steffisburg"
1,1733470799-2,CARADO I 449 Édition 15 Fiat Capron 169L/1449,CHF 71'900.–,14'000 km,Diesel,"Zone industrielle Les Portettes 2c, 1312 Eclépens"
2,1733470801-3,DETHLEFFS Fiat Ducato,CHF 37'800.–,90'000 km,Diesel,"Rte du St-Bernard 8, 1937 Orsières"
3,1733470805-4,FIAT Euro Master Wohnmobil,CHF 19'900.–,187'236 km,Diesel,"Badenerstrasse 84, 8952 Schlieren"
4,1733470807-5,VW T6.1 California 2.0 TDI Ocean Edition Liber...,CHF 84'500.–,16'500 km,Diesel,8852 Altendorf


### Count numbre of rows and columns in the dateframe

In [429]:
# Dimension (rows, columns)
print('Dimension:', df.shape)

# Number of rows
print('Number of rows:', df.shape[0])

# Number of columns
print('Number of columns:', df.shape[1])

Dimension: (1587, 6)
Number of rows: 1587
Number of columns: 6


### Get Data types from Web scraping

In [430]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

web-scraper-order    object
description_raw      object
price_raw            object
mileage_raw          object
fuel_type_raw        object
location_raw         object
dtype: object

#### Extract and save relevant information from raw data using regular expressions (regex)

### Extract Price

In [431]:
# Extract values from 'price_raw' column
price = []
for i in df['price_raw']:
    d1 = re.findall(r'[0-9]+', str(i))
    try:
        d2 = ''.join(d1).replace("'", "").replace("CHF", "").replace(".-", "").strip()
    except:
        d2 = None
    price.append(d2)

# Save as new variable in the pandas data frame
df['price'] = pd.Series(price, dtype="int")
    
# Print first 5 values
print(df[['price_raw', 'price']].head(5))

      price_raw  price
0  CHF 15'900.–  15900
1  CHF 71'900.–  71900
2  CHF 37'800.–  37800
3  CHF 19'900.–  19900
4  CHF 84'500.–  84500


In [432]:
df.dtypes

web-scraper-order    object
description_raw      object
price_raw            object
mileage_raw          object
fuel_type_raw        object
location_raw         object
price                 int64
dtype: object

In [433]:
df.head()

,web-scraper-order,description_raw,price_raw,mileage_raw,fuel_type_raw,location_raw,price
0,1733470796-1,MERCEDES-BENZ Camper,CHF 15'900.–,265'000 km,Diesel,"Mittelstrasse 38, 3613 Steffisburg",15900
1,1733470799-2,CARADO I 449 Édition 15 Fiat Capron 169L/1449,CHF 71'900.–,14'000 km,Diesel,"Zone industrielle Les Portettes 2c, 1312 Eclépens",71900
2,1733470801-3,DETHLEFFS Fiat Ducato,CHF 37'800.–,90'000 km,Diesel,"Rte du St-Bernard 8, 1937 Orsières",37800
3,1733470805-4,FIAT Euro Master Wohnmobil,CHF 19'900.–,187'236 km,Diesel,"Badenerstrasse 84, 8952 Schlieren",19900
4,1733470807-5,VW T6.1 California 2.0 TDI Ocean Edition Liber...,CHF 84'500.–,16'500 km,Diesel,8852 Altendorf,84500


### Extract Mileage

In [434]:
# Extract values from 'mileage_raw' column
mileage = []
for i in df['mileage_raw']:
    d1 = re.findall(r'[0-9]+', str(i))
    try:
        d2 = ''.join(d1).replace("'", "").replace("km", "").replace(".-", "").strip()
    except:
        d2 = None
    mileage.append(d2)

# Save as new variable in the pandas data frame
df['mileage'] = pd.Series(mileage, dtype="int")
    
# Print first 5 values
print(df[['mileage_raw','mileage']].head(5))

  mileage_raw  mileage
0  265'000 km   265000
1   14'000 km    14000
2   90'000 km    90000
3  187'236 km   187236
4   16'500 km    16500


In [435]:
df.dtypes

web-scraper-order    object
description_raw      object
price_raw            object
mileage_raw          object
fuel_type_raw        object
location_raw         object
price                 int64
mileage               int64
dtype: object

### Fuel Type to Dummy

In [436]:
#count frequency of values in column 'fuel_type_raw'
df['fuel_type_raw'].value_counts()

fuel_type_raw
Diesel     1453
Benzin       72
-            61
Elektro       1
Name: count, dtype: int64

In [437]:
# Filter für Diesel und Benzin
df = df[(df['fuel_type_raw'] == 'Diesel') | (df['fuel_type_raw'] == 'Benzin')]

count = df['fuel_type_raw'].value_counts()
print(count)

# One-Hot-Encoding für die Spalte 'fuel_type_raw'
df_encoded = pd.get_dummies(df['fuel_type_raw'], prefix='fuel_type', drop_first=True)

# Konvertiere boolesche Werte in 0 und 1 (falls vorhanden)
bool_columns = df_encoded.select_dtypes(include=['bool']).columns
df_encoded[bool_columns] = df_encoded[bool_columns].astype(int)

# Füge nur die neuen One-Hot-encoded Spalten dem ursprünglichen DataFrame hinzu
df = df.join(df_encoded)

# Überprüfe das Ergebnis
df.head()

fuel_type_raw
Diesel    1453
Benzin      72
Name: count, dtype: int64


,web-scraper-order,description_raw,price_raw,mileage_raw,fuel_type_raw,location_raw,price,mileage,fuel_type_Diesel
0,1733470796-1,MERCEDES-BENZ Camper,CHF 15'900.–,265'000 km,Diesel,"Mittelstrasse 38, 3613 Steffisburg",15900,265000,1
1,1733470799-2,CARADO I 449 Édition 15 Fiat Capron 169L/1449,CHF 71'900.–,14'000 km,Diesel,"Zone industrielle Les Portettes 2c, 1312 Eclépens",71900,14000,1
2,1733470801-3,DETHLEFFS Fiat Ducato,CHF 37'800.–,90'000 km,Diesel,"Rte du St-Bernard 8, 1937 Orsières",37800,90000,1
3,1733470805-4,FIAT Euro Master Wohnmobil,CHF 19'900.–,187'236 km,Diesel,"Badenerstrasse 84, 8952 Schlieren",19900,187236,1
4,1733470807-5,VW T6.1 California 2.0 TDI Ocean Edition Liber...,CHF 84'500.–,16'500 km,Diesel,8852 Altendorf,84500,16500,1


### Count and identify missing values

In [438]:
# Count missing values
print(pd.isna(df).sum())

# Identify rows with missing values
df[df.isna().any(axis=1)].head()

web-scraper-order    0
description_raw      0
price_raw            0
mileage_raw          0
fuel_type_raw        0
location_raw         0
price                0
mileage              0
fuel_type_Diesel     0
dtype: int64


,web-scraper-order,description_raw,price_raw,mileage_raw,fuel_type_raw,location_raw,price,mileage,fuel_type_Diesel


## Count and identify duplicated values (if any)

In [439]:
# Count duplicated values
print(df.duplicated().sum())

# Identify rows with duplicated values, e.g.:
df[df[['web-scraper-order', 'description_raw', 'price_raw', 'mileage_raw' ]].duplicated()]

0


,web-scraper-order,description_raw,price_raw,mileage_raw,fuel_type_raw,location_raw,price,mileage,fuel_type_Diesel


## Save to Datafile

In [440]:
df.to_csv('./data/Camper_data_prepared.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)

In [441]:
df.describe()

,price,mileage,fuel_type_Diesel
count,1525.000000,1525.000000,1525.000000
mean,56041.168525,74055.251148,0.952787
std,38267.724371,64241.779780,0.212164
min,900.000000,10000.000000,0.000000
25%,37900.000000,27500.000000,1.000000
50%,53900.000000,53000.000000,1.000000
75%,65997.000000,100500.000000,1.000000
max,499000.000000,450000.000000,1.000000


## Create SQLite Database

In [442]:
# Create a connection to the database
conn = sqlite3.connect('./data/Camper.db')

# Check if the connection was successful
check_conn(conn)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Show dbs in the directory
flist = fnmatch.filter(os.listdir('.'), '*.db')
for i in flist:
    print(i)

## Create SQL-Table in the database

In [443]:
cursor.execute("DROP TABLE IF EXISTS Camper")

# Create a table
cursor.execute("""CREATE TABLE Camper (
                web_scraper_order TEXT,
                description_raw TEXT,
                price_raw TEXT,
                mileage_raw TEXT,
                location_raw TEXT,
                price INTEGER,
                mileage INTEGER,
                fuel_type_raw TEXT,
                fuel_type_Diesel INTEGER
                )""")

# Commit the changes
conn.commit()

# Read the data from the CSV file
df = pd.read_csv('./data/Camper_data_prepared.csv', 
                 sep=",", 
                 encoding='utf-8')[['web-scraper-order',
                                    'description_raw',
                                    'price_raw', 
                                    'mileage_raw',
                                    'fuel_type_raw', 
                                    'location_raw',
                                    'price', 
                                    'mileage',  
                                    'fuel_type_Diesel']]
print(df.shape)


(1525, 9)


In [444]:
df.head(5)

,web-scraper-order,description_raw,price_raw,mileage_raw,fuel_type_raw,location_raw,price,mileage,fuel_type_Diesel
0,1733470796-1,MERCEDES-BENZ Camper,CHF 15'900.–,265'000 km,Diesel,"Mittelstrasse 38, 3613 Steffisburg",15900,265000,1
1,1733470799-2,CARADO I 449 Édition 15 Fiat Capron 169L/1449,CHF 71'900.–,14'000 km,Diesel,"Zone industrielle Les Portettes 2c, 1312 Eclépens",71900,14000,1
2,1733470801-3,DETHLEFFS Fiat Ducato,CHF 37'800.–,90'000 km,Diesel,"Rte du St-Bernard 8, 1937 Orsières",37800,90000,1
3,1733470805-4,FIAT Euro Master Wohnmobil,CHF 19'900.–,187'236 km,Diesel,"Badenerstrasse 84, 8952 Schlieren",19900,187236,1
4,1733470807-5,VW T6.1 California 2.0 TDI Ocean Edition Liber...,CHF 84'500.–,16'500 km,Diesel,8852 Altendorf,84500,16500,1


## Write data to the SQL-table in data base

In [445]:
df.to_sql(name = 'campervans_table',
          con = conn,
          index = False,
          if_exists = 'replace')

1525

## Query the SQL-table

In [451]:
# Query the SQL-table
cursor.execute('''SELECT *
               FROM campervans_table
               WHERE price >=56041''')

df = pd.DataFrame(cursor.fetchall(), 
                  columns=['web-scraper-order',
                                    'description_raw',
                                    'price_raw', 
                                    'mileage_raw',
                                    'fuel_type_raw', 
                                    'location_raw',
                                    'price', 
                                    'mileage',  
                                    'fuel_type_Diesel'])
df.head()

,web-scraper-order,description_raw,price_raw,mileage_raw,fuel_type_raw,location_raw,price,mileage,fuel_type_Diesel
0,1733470799-2,CARADO I 449 Édition 15 Fiat Capron 169L/1449,CHF 71'900.–,14'000 km,Diesel,"Zone industrielle Les Portettes 2c, 1312 Eclépens",71900,14000,1
1,1733470807-5,VW T6.1 California 2.0 TDI Ocean Edition Liber...,CHF 84'500.–,16'500 km,Diesel,8852 Altendorf,84500,16500,1
2,1733470809-6,"VW T6.1 Knaus Tourer Van 500 MW, MEGA Ausstatt...",CHF 69'900.–,11'000 km,Diesel,8852 Altendorf,69900,11000,1
3,1733470812-7,VW T6.1 California 2.0 TDI Ocean Liberty,CHF 56'500.–,45'000 km,Diesel,"Güterstrasse 37, 9444 Diepoldsau",56500,45000,1
4,1733470815-8,HYMER BMC 580-I,CHF 115'800.–,12'200 km,Diesel,"Büelhalde 4, 8317 Tagelswangen",115800,12200,1
